In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-07-02 20:22:46--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  59.3MB/s    in 0.4s    

2023-07-02 20:22:47 (59.3 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [32]:
# clean the data

user_ratings = df_ratings['user'].value_counts()
book_ratings = df_ratings['isbn'].value_counts()

# display(user_ratings)
# display(book_ratings)

df_books.dropna(inplace=True)
df_books.dropna(inplace=True)

filtered_df = df_ratings[(df_ratings['user'].isin(user_ratings[user_ratings >= 200].index)) & (df_ratings['isbn'].isin(book_ratings[book_ratings >= 100].index))]
# display(filtered_df)

filtered_df = filtered_df.pivot_table(index=['isbn'],columns=['user'],values='rating').fillna(0)
# display(filtered_df)

filtered_df.index = filtered_df.join(df_books.set_index('isbn'))['title'].sort_index()
# display(filtered_df)

In [30]:
model = NearestNeighbors(metric='cosine')
model.fit(filtered_df.values)

NearestNeighbors(metric='cosine')

In [41]:
from pandas.core.groupby.generic import reconstruct_func
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  if book in filtered_df.index:
    curr_title = filtered_df.loc[book]
    distance, index = model.kneighbors([curr_title.values], n_neighbors=6)
    recs = [book, pd.DataFrame({'title': filtered_df.iloc[index[0]].index.values, 'distance': distance[0]}).sort_values(by='distance', ascending=False).head(5).values]
    return recs
  else:
    print("Please type a valid book title")
    return

In [42]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", array([["I'll Be Seeing You", 0.8016210794448853],
       ['The Weight of Water', 0.7708583474159241],
       ['The Surgeon', 0.7699410915374756],
       ['I Know This Much Is True', 0.7677075266838074],
       ['The Lovely Bones: A Novel', 0.7234864234924316]], dtype=object)]
You passed the challenge! 🎉🎉🎉🎉🎉
